In [1]:
from pyscf import gto  # PySCF is a quantum chemistry python module
import SCF
import numpy as np

/opt/anaconda3/envs/venv/lib/python3.8/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


In [2]:
# Input Data
mol_h2o = gto.M(unit="Bohr",
                atom="O 0.000000000000  -0.143225816552   0.000000000000;"
                + "H 1.638036840407   1.136548822547  -0.000000000000;"
                + "H -1.638036840407   1.136548822547  -0.000000000000",
                basis='STO-3g')
mol_h2o.build()

# Convergence Criteria
E_conv_threshold = 1.0E-10
D_conv_threshold = 1.0E-8
max_iterations = 1000

# get the integrals
Suv = mol_h2o.intor('int1e_ovlp')  # Overlap Integrals
Tuv = mol_h2o.intor('int1e_kin')  # Kinetic Energy 1 electron integrals
Vuv = mol_h2o.intor('int1e_nuc')  # Nuclear Repulsion 1 electron integrals
eri = mol_h2o.intor("int2e")  # Electron Repulsion 2 electron integrals

In [3]:
print(Tuv.shape)
print(Vuv.shape)

print(eri.shape) # 4-D, electron1, electron2, spin1, spin2

(7, 7)
(7, 7)
(7, 7, 7, 7)


In [4]:
Enuc = SCF.calc_nuclear_repulsion_energy(mol_h2o)
print(Enuc)

8.00236706181077


In [5]:
Huv = SCF.calc_hcore_matrix(Tuv, Vuv)
print(Huv[0, 0], Huv[3, 4], Huv[4, 3])
print(Huv.shape)

-32.57739541261037 0.0 0.0
(7, 7)


In [6]:
Duv = SCF.calc_initial_density(mol_h2o)
print(Duv)
print(Duv.shape)

[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]
(7, 7)


In [7]:
for it in range(max_iterations):
    Fuv = SCF.calc_fock_matrix(mol_h2o, Huv, eri, Duv)
    mo_e, mo_c = SCF.solve_Roothan_equations(Fuv, Suv)
    
    Etot_new = SCF.calc_tot_energy(Fuv, Huv, Duv, Enuc)
    Duv_new = SCF.form_density_matrix(mol_h2o, mo_c)
    break

In [8]:
print(Fuv[0, 0])
print(Fuv[2, 5])
print(Fuv[5, 2])

-32.57739541261037
-1.6751501447185013
-1.6751501447185015


In [9]:
print(mo_e)
print()
print(mo_c[0, :])
print(mo_e.shape, mo_c.shape)

[-32.57830292  -8.08153571  -7.55008599  -7.36396923  -7.34714487
  -4.00229867  -3.98111115]

[-1.00154358e+00 -2.33624458e-01  3.49050635e-16 -8.56842145e-02
 -1.10314884e-29  4.82226067e-02 -1.11404715e-15]
(7,) (7, 7)


In [10]:
print(Etot_new)

8.00236706181077


In [11]:
print(Duv_new[0, 0])
print(Duv_new[2, 5], Duv_new[5, 2])

2.130023428655503
-0.292263302096531 -0.292263302096531


In [12]:
arr = np.random.random((7, 7, 7, 7))
arr

array([[[[6.97656519e-01, 8.06568116e-01, 7.31153791e-01, ...,
          8.05535388e-01, 3.58607477e-01, 3.63851063e-01],
         [6.61197455e-01, 4.08343414e-01, 8.88544966e-02, ...,
          9.10110310e-01, 3.64975679e-01, 4.23531410e-01],
         [8.86304198e-01, 9.80012748e-01, 2.70457182e-01, ...,
          9.34859671e-01, 7.48598711e-01, 6.54368750e-01],
         ...,
         [5.17176266e-01, 8.11070133e-02, 2.91836500e-01, ...,
          2.42517308e-02, 6.26346488e-01, 5.82894533e-01],
         [6.37401817e-01, 5.39988999e-01, 4.83395332e-03, ...,
          5.90872210e-01, 9.11928664e-01, 9.74329151e-01],
         [9.85898114e-01, 2.22755584e-01, 9.81583599e-01, ...,
          6.75539626e-01, 2.64379803e-01, 1.78240416e-02]],

        [[5.99872125e-01, 2.86747766e-02, 3.65706570e-01, ...,
          8.86837162e-01, 7.72916213e-01, 5.85445451e-01],
         [6.76222379e-04, 4.88422230e-01, 9.18758966e-01, ...,
          9.95886848e-02, 7.86350707e-01, 9.14262493e-01],
        

In [13]:
arr1 = arr[0, 0]
arr1

array([[0.69765652, 0.80656812, 0.73115379, 0.5084824 , 0.80553539,
        0.35860748, 0.36385106],
       [0.66119746, 0.40834341, 0.0888545 , 0.88598421, 0.91011031,
        0.36497568, 0.42353141],
       [0.8863042 , 0.98001275, 0.27045718, 0.36550811, 0.93485967,
        0.74859871, 0.65436875],
       [0.84120676, 0.64764576, 0.0442256 , 0.30809523, 0.34569154,
        0.09717011, 0.90725934],
       [0.51717627, 0.08110701, 0.2918365 , 0.81516248, 0.02425173,
        0.62634649, 0.58289453],
       [0.63740182, 0.539989  , 0.00483395, 0.50921929, 0.59087221,
        0.91192866, 0.97432915],
       [0.98589811, 0.22275558, 0.9815836 , 0.40475614, 0.67553963,
        0.2643798 , 0.01782404]])

In [14]:
arr1.shape

(7, 7)

In [15]:
arr2 = arr[0, :, 0]
arr2.shape

(7, 7)

In [16]:
arr2

array([[0.69765652, 0.80656812, 0.73115379, 0.5084824 , 0.80553539,
        0.35860748, 0.36385106],
       [0.59987213, 0.02867478, 0.36570657, 0.17394241, 0.88683716,
        0.77291621, 0.58544545],
       [0.85340534, 0.22181846, 0.09391628, 0.21686106, 0.51026421,
        0.68393628, 0.15899366],
       [0.90198214, 0.90698108, 0.8905761 , 0.0635448 , 0.61728412,
        0.94856171, 0.50528575],
       [0.39273616, 0.10852694, 0.28169542, 0.46029658, 0.83259413,
        0.73350791, 0.17766341],
       [0.53469265, 0.59497338, 0.25958893, 0.89130868, 0.8527093 ,
        0.42282799, 0.49818945],
       [0.82562297, 0.69978698, 0.33274303, 0.08469533, 0.17829474,
        0.37170471, 0.03209836]])